# Basic imports

In [30]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import DocArrayInMemorySearch

from operator import itemgetter

In [27]:
MODEL = 'llama3'

ollama_model = Ollama(model=MODEL)
ollama_embeddings = OllamaEmbeddings(model=MODEL)

In [5]:
ollama_model.invoke('Tell me a joke')

"Here's one:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!\n\nHope that made you smile!"

In [7]:
parser = StrOutputParser()

chain = ollama_model | parser
chain.invoke('Tell me a joke')

"Here's one:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!\n\nHope that made you smile!"

In [25]:
file = 'Resources/2024_Kosar_Computer science education in chatgpt era.pdf'

loader = PyPDFLoader(file)

pages = loader.load_and_split()

In [18]:
prompt_template = '''
You are a research assistant that is tasked with reviewing scientific papers. You are given different research questions that you have to answer by reading the papers. You will be given one paper at a time. You have to read the paper and answer the question based on the content of the paper. Do not add information not present in the paper.

Context: {context}

Question: {question}
'''

In [32]:
prompt = PromptTemplate.from_template(prompt_template)
print(prompt.format(context='Here is some context', question='Here is a question'))


You are a research assistant that is tasked with reviewing scientific papers. You are given different research questions that you have to answer by reading the papers. You will be given one paper at a time. You have to read the paper and answer the question based on the content of the paper. Do not add information not present in the paper.

Context: Here is some context

Question: Here is a question



In [23]:
chain = prompt | ollama_model | parser

In [29]:
vectore_store = DocArrayInMemorySearch.from_documents(pages, embedding = ollama_embeddings)

retriever = vectore_store.as_retriever()

e:\Programs\miniconda3\envs\research_ass\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [33]:
chain = (
    {
        'context': itemgetter('question') | retriever,
        'question': itemgetter('question')
    }
    | prompt
    | ollama_model
    | parser
)

In [34]:
questions = [
    'What is the contribution of the paper?',
    'What are the implications for programming education?',
    'What data has been used in the paper?',
    'What are the advantages and disadvantages of the proposed approach?'
]

In [36]:
for question in questions:
    print(f'Question: {question}')
    print(f"Answer: {chain.invoke({'question': question})}")
    print('-------------------')

Question: What is the contribution of the paper?
Answer: Based on the provided paper, it appears to be a study on the use and effectiveness of ChatGPT in computer science education. The paper presents the results of an experiment that involved two groups of students: one group used ChatGPT for practical assignments, while the other group did not.

The main contributions of this paper are:

1. Insights into how students use ChatGPT: The study shows that most students used ChatGPT for code optimization and comparison with their own code, rather than generating new code.
2. Code understandability: The results suggest that the code generated by ChatGPT was generally understandable to students who used it.
3. Influence of ChatGPT on exam grades: Despite concerns about over-reliance on ChatGPT, the study found no significant impact on students' overall grades.

Overall, this paper provides practical insights into the use and effectiveness of ChatGPT in computer science education, which can i